In [1]:
import torch
import torch.nn.functional as fun
import numpy as np

from google_play_scraper import Sort, reviews
import csv

from nltk.tokenize import wordpunct_tokenize

def clean_word(sentence):
    sentence = wordpunct_tokenize(sentence)
    cleaned = []
    for word in sentence:
        if(word.isalpha()):
            cleaned.append(word.lower())
    return cleaned

def get_corpus_from_csv():
    corpus = []
    with open("dfyutup-cleanedfixversion3.csv") as file_buffer:
        csv_reader = list(csv.reader(file_buffer))
        n = len(csv_reader)
        for i in range(1, n):
            corpus.append(clean_word(csv_reader[i][2]))
    return corpus

    
def get_corpus_google_play(cnt = 100):
    result, _ = reviews(
        'com.apps.MyXL',
        lang='id', 
        country='id', 
        sort=Sort.MOST_RELEVANT, 
        count=cnt, 
        filter_score_with=None
    )
    return [clean_word(konten["content"]) for konten in result]

# print(get_corpus_google_play())
# print(get_corpus_from_csv())

corpus = get_corpus_from_csv()
# print(corpus)

In [7]:
import torch
import torch.nn.functional as fun
import numpy as np


def create_vocab(corpus, limit_occ = 50):
    """
    membuat vocabulary, dengan 2 info penting: 
        1. word_id -> mapping kata ke id (int)
        2. id_word -> sebaliknya, mapping dari id ke word
    """
    word_id = {}
    id_word = []

    occurence = dict()
    i = 0
    for sentence in corpus:
        for word in sentence:
            occurence.setdefault(word, 0)
            occurence[word] += 1
            
    for k, v in occurence.items():
        if(v < limit_occ): continue
        print(k)
        word_id[k] = i
        id_word.append(k)
        i += 1
                
    id_word = {id: word for (word, id) in word_id.items()}
    return word_id, id_word


word_id, id_word = create_vocab(corpus)
vocab_size = len(word_id)

print(word_id)


rick
lu
anak
ya
itu
bang
gw
sub
kontol
apa
di
mu
muka
lo
anjing
gue
nonton
youtube
suka
gaya
haha
wkwkwkw
buat
haters
semua
sok
video
erickolim
kalo
gk
ngapain
ditonton
eriko
juga
dia
klo
gak
cuma
doang
kayak
memek
asu
babi
kecil
v
tapi
ngakak
ericko
lim
walaupun
toxic
mantap
dah
rik
tuh
aja
gua
konten
adsense
masih
iya
banget
anjeng
bikin
vidio
ada
yg
kali
dari
goblok
ni
bgt
kek
org
subscribe
trus
komentar
lagi
biar
hater
kagak
terus
harus
semangat
aku
tau
terkenal
bukan
hal
yang
dan
tetap
eric
jadi
youtuber
lebih
sukses
bisa
erick
fans
karna
kalau
jangan
bau
karya
sabar
ngomong
usah
i
love
you
auto
subs
like
bilang
sama
orang
jembut
otak
bg
para
tu
banyak
bro
menghibur
anti
si
mah
k
ajg
wkwk
nya
tetep
salah
ganteng
penting
berkarya
comment
intro
emang
channel
tanpa
bakal
ini
hahaha
heters
tolol
hiburan
mendidik
is
the
duit
btw
d
jgn
tpi
kan
bocah
njing
satu
mati
kapan
komen
bangsat
eh
subscriber
atau
lain
sangat
karena
bego
punya
mending
kyk
gini
kasar
sumpah
gila
pada
koment
lah
eri

In [14]:
# dataset generator
def iter_dataset(corpus, context_window=1):
    for sentence in corpus:
        words = sentence
        for i in range(len(words)):
            center_word = words[i]
            # cari konteks ke kiri
            j = i - 1
            while (j >= i - context_window) and (j >= 0):
                context_word = words[j]
                yield (center_word, context_word)
                j -= 1
            # cari konteks ke kanan
            j = i + 1
            while (j <= i + context_window) and (j < len(words)):
                context_word = words[j]
                yield (center_word, context_word)
                j += 1


def get_dataset(corpus):
    # membuat dataset
    X = []
    Y = []
    for center, context in iter_dataset(corpus):
        try:
            if(center not in word_id.keys() or context not in word_id.keys()):
                continue
            X.append(word_id[center])
            Y.append(word_id[context])
        except:
            pass

    X_ctr = torch.tensor(X)
    Y_ctx = torch.tensor(Y)
    return X_ctr, Y_ctx


def get_input_tensor(tensor):
    return fun.one_hot(tensor, num_classes=vocab_size)


center_list, context_list = get_dataset(corpus)
# get_input_tensor(center_list[2])


In [15]:
# Softmax
def softmax(x):
    maxes = torch.max(x, 1, keepdim=True)[0]
    x_exp = torch.exp(x - maxes)
    x_exp_sum = torch.sum(x_exp, 1, keepdim=True)
    return x_exp / x_exp_sum

# Categorical Cross Entropy Loss


def CCEloss(Y_pred, Y_true):
    m = Y_pred.size()[0]
    return -(1 / m) * torch.sum(Y_true * torch.log(Y_pred))


In [ ]:
tmp_c = None
tmp_w = None

In [19]:

def train(X_ctr, Y_ctx, EPOCHS = 200, load = False):
    global tmp_w
    global tmp_c

    EMBEDDING_DIMS = 5

    initrange = 0.5 / EMBEDDING_DIMS

    # trick: di awal, mode gradient jangan diaktifkan dahulu, karena ingin
    # memodifikasi nilai di C dan W secara in-place
    C = torch.rand(vocab_size, EMBEDDING_DIMS, requires_grad = False)
    W = torch.rand(EMBEDDING_DIMS, vocab_size, requires_grad = False)

    # agar nilai awal parameter di C dan W berkisar di antara -initrange hingga +initrange
    C = -2 * initrange * C + initrange
    W = -2 * initrange * W + initrange

    # setelah operasi in-place modification, baru kita set mode gradient
    C.requires_grad = True
    W.requires_grad = True
    if(load):
        C = tmp_c
        W = tmp_w

    print(f'C shape is: {C.shape}, W shape is: {W.shape}')
    print(C)
    print(W)

    LEARNING_RATE = 0.2
    LR_DECAY = 0.99

    for i in range(EPOCHS):
        X = get_input_tensor(X_ctr).float()
        Y = get_input_tensor(Y_ctx).float()

        h = X.mm(C)
        Y_pred = softmax(h.mm(W))

        loss = CCEloss(Y_pred, Y)
        loss.backward()

        # update C dan W
        with torch.no_grad():
            C -= LEARNING_RATE * C.grad
            W -= LEARNING_RATE * W.grad
            
            C.grad.zero_()
            W.grad.zero_()
        tmp_w = W
        tmp_c = C
        if i % 10 == 0:
            # kita coba lihat progress cosine similarity
            # antara embedding "lo" dengan "lu"
            vector_1 = C[word_id["lo"]]
            vector_2 = C[word_id["lu"]]
            sim = fun.cosine_similarity(vector_1, vector_2, dim = 0)
            print(f'Epoch {i}, loss = {loss}, sim(lo, lu) = {sim}')
    return C, W
# embedding_C, context_W = train(center_list, context_list, True)


C shape is: torch.Size([409, 5]), W shape is: torch.Size([5, 409])
tensor([[-0.0995,  0.0102, -0.0447, -0.0807, -0.0814],
        [ 0.0610,  0.0906, -0.0281, -0.0176, -0.0089],
        [ 0.0777,  0.0550, -0.0556, -0.0927, -0.0088],
        ...,
        [ 0.0422, -0.0273,  0.0035, -0.0918, -0.0574],
        [-0.0695, -0.0651, -0.0851, -0.0298, -0.0688],
        [ 0.0846, -0.0459,  0.0214,  0.0025, -0.0558]], requires_grad=True)
tensor([[-0.0534, -0.0683, -0.0326,  ...,  0.0691, -0.0342, -0.0645],
        [-0.0272,  0.0379,  0.0603,  ..., -0.0853,  0.0342,  0.0274],
        [-0.0303,  0.0429,  0.0806,  ...,  0.0990, -0.0245,  0.0684],
        [-0.0182, -0.0391, -0.0305,  ...,  0.0790,  0.0973, -0.0115],
        [-0.0230, -0.0438,  0.0288,  ..., -0.0514,  0.0224, -0.0278]],
       requires_grad=True)
Epoch 0, loss = 6.013712406158447, sim(lo, lu) = 0.45380493998527527
Epoch 10, loss = 6.0135297775268555, sim(lo, lu) = 0.46046119928359985
Epoch 20, loss = 6.013360023498535, sim(lo, lu) = 0

In [23]:

embedding_C, context_W = train(center_list, context_list, 500, True)

C shape is: torch.Size([409, 5]), W shape is: torch.Size([5, 409])
tensor([[-0.1011,  0.0102, -0.0417, -0.0814, -0.0808],
        [ 0.0655,  0.0832, -0.0152, -0.0335,  0.0027],
        [ 0.0767,  0.0557, -0.0519, -0.0920, -0.0075],
        ...,
        [ 0.0424, -0.0278,  0.0042, -0.0919, -0.0576],
        [-0.0770, -0.0594, -0.0912, -0.0120, -0.0661],
        [ 0.0838, -0.0447,  0.0229,  0.0014, -0.0541]], requires_grad=True)
tensor([[-0.0520, -0.0553, -0.0318,  ...,  0.0688, -0.0478, -0.0658],
        [-0.0253,  0.0357,  0.0635,  ..., -0.0839,  0.0244,  0.0293],
        [-0.0278,  0.0495,  0.0792,  ...,  0.0991, -0.0404,  0.0693],
        [-0.0199, -0.0578, -0.0322,  ...,  0.0801,  0.0944, -0.0116],
        [-0.0225, -0.0444,  0.0291,  ..., -0.0516,  0.0102, -0.0304]],
       requires_grad=True)
Epoch 0, loss = 5.994383811950684, sim(lo, lu) = 0.5948783159255981
Epoch 10, loss = 5.990130424499512, sim(lo, lu) = 0.6020410060882568
Epoch 20, loss = 5.984925270080566, sim(lo, lu) = 0.60

In [24]:
embedding_C, context_W = train(center_list, context_list, 500, True)

C shape is: torch.Size([409, 5]), W shape is: torch.Size([5, 409])
tensor([[-0.1022,  0.0120, -0.0320, -0.0854, -0.0790],
        [ 0.0872,  0.0722,  0.0290, -0.0843,  0.0346],
        [ 0.0754,  0.0591, -0.0420, -0.0912, -0.0042],
        ...,
        [ 0.0430, -0.0286,  0.0058, -0.0921, -0.0581],
        [-0.1071, -0.0502, -0.1193,  0.0332, -0.0685],
        [ 0.0819, -0.0415,  0.0268, -0.0012, -0.0499]], requires_grad=True)
tensor([[-0.0470, -0.0240, -0.0296,  ...,  0.0681, -0.0894, -0.0689],
        [-0.0172,  0.0297,  0.0738,  ..., -0.0773,  0.0067,  0.0371],
        [-0.0190,  0.0773,  0.0760,  ...,  0.0989, -0.0873,  0.0712],
        [-0.0253, -0.1133, -0.0356,  ...,  0.0845,  0.1030, -0.0103],
        [-0.0195, -0.0450,  0.0306,  ..., -0.0518, -0.0193, -0.0367]],
       requires_grad=True)
Epoch 0, loss = 5.704797744750977, sim(lo, lu) = 0.8610402345657349
Epoch 10, loss = 5.704591751098633, sim(lo, lu) = 0.864213228225708
Epoch 20, loss = 5.704378604888916, sim(lo, lu) = 0.867

In [25]:
embedding_C, context_W = train(center_list, context_list, 500, True)

C shape is: torch.Size([409, 5]), W shape is: torch.Size([5, 409])
tensor([[-0.1000,  0.0135, -0.0183, -0.0951, -0.0765],
        [ 0.1282,  0.0605,  0.0980, -0.1647,  0.0721],
        [ 0.0752,  0.0631, -0.0308, -0.0927, -0.0008],
        ...,
        [ 0.0439, -0.0294,  0.0076, -0.0925, -0.0586],
        [-0.1604, -0.0494, -0.1729,  0.0868, -0.0849],
        [ 0.0800, -0.0384,  0.0309, -0.0039, -0.0459]], requires_grad=True)
tensor([[-0.0437,  0.0188, -0.0311,  ...,  0.0625, -0.1540, -0.0769],
        [-0.0139,  0.0247,  0.0802,  ..., -0.0755, -0.0132,  0.0402],
        [-0.0088,  0.1311,  0.0708,  ...,  0.0942, -0.1564,  0.0692],
        [-0.0371, -0.1912, -0.0406,  ...,  0.0888,  0.1334, -0.0093],
        [-0.0144, -0.0443,  0.0327,  ..., -0.0519, -0.0530, -0.0433]],
       requires_grad=True)
Epoch 0, loss = 5.672426223754883, sim(lo, lu) = 0.9519789218902588
Epoch 10, loss = 5.670852184295654, sim(lo, lu) = 0.9528663158416748
Epoch 20, loss = 5.669210910797119, sim(lo, lu) = 0.95

In [28]:
import pickle

# obj0, obj1, obj2 are created here...

def save_pickle(idx = 0):
    global tmp_c, tmp_w
    # Saving the objects:
    with open(f'objs-{idx}.pkl', 'wb') as f:
        pickle.dump([tmp_c, tmp_w], f)

def load_pickle(idx = 0):
    global tmp_c, tmp_w
    # Getting back the objects:
    with open(f'objs-{idx}.pkl', 'rb') as f:
        tmp_c, tmp_w = pickle.load(f)

In [29]:
save_pickle(1)

In [ ]:
def get_diff(word_1, word_2):
    vector_1 = embedding_C[word_id[word_1]]
    vector_2 = embedding_C[word_id[word_2]]
    print(fun.cosine_similarity(vector_1, vector_2, dim = 0))



def show_vector(word_1):
    vector_1 = embedding_C[word_id[word_1]].detach().numpy()
    print(vector_1)

get_diff("kek", "kayak")